## Context

The purpose of this notebook is to compare sklearn current nearest neighbors heuristic with a new proposal. It assumes readers have previously taken a look at the benchmark analyses notebook, [sklearn_nn_heuristic.ipynb](https://nbviewer.jupyter.org/github/gbolmier/sklearn-neighbors-benchmark/blob/master/sklearn_nn_heuristic.ipynb).

This comparison focus on both side worst case scenarios. The current heuristic ordinarily consists of choosing `'kd-tree'` when `n_neighbors < n_samples // 2`, otherwise `'brute'`. In practice, that often (if not always) mean going for `'kd_tree'`. On the other hand, the proposed heuristic, suggest to switch for `'brute'` when `n_features > 15`, otherwise `'kd_tree'`.

Extreme data structure scenarios are settled to show by how much each heuristic can possibly fail:
- [I - Low intrinsic dimensionality and noise](#I---Low-intrinsic-dimensionality-and-noise)
- [II - High intrinsic dimensionality](#II---High-intrinsic-dimensionality)

Datasets are all synthetic and generated by sampling standard normal data. Noise is created by dividing concerned features by a factor of 1,000.

In [1]:
import numpy as np

from sklearn.neighbors import NearestNeighbors

Define datasets and benchmarking utilities.

In [2]:
N_SAMPLES_TRAIN = 50_000
N_SAMPLES_TEST = 50_000

N_NEIGHBORS = 10

def get_standard_normal(n_features, random_state=32):
    np.random.seed(random_state)
    X_train = np.random.randn(N_SAMPLES_TRAIN, n_features)
    X_test = np.random.randn(N_SAMPLES_TEST, n_features)
    return X_train, X_test


def get_low_intrinsic_dim_and_noise(intrinsic_dim, noise_dim, random_state=32):
    np.random.seed(random_state)
    X_signal = np.random.randn(N_SAMPLES_TRAIN, intrinsic_dim)
    X_noise = np.random.randn(N_SAMPLES_TRAIN, noise_dim) / 1_000
    X = np.hstack((X_signal, X_noise))
    X_train, X_test = X[:N_SAMPLES_TRAIN], X[-N_SAMPLES_TEST:]
    return X_train, X_test


def choose_algo(X, heuristic):
    n_samples, n_features = X.shape
    
    if heuristic == 'current':
        algorithm = 'kd_tree' if N_NEIGHBORS < n_samples // 2 else 'brute'
        
    if heuristic == 'proposed':
        algorithm = 'brute' if n_features > 15 else 'kd_tree'
    
    return algorithm


def fit_predict(X_train, x_test, model):
    model.fit(X_train)
    model.kneighbors(X_test, return_distance=False)

## I - Low intrinsic dimensionality and noise

In [3]:
X_train, X_test = get_low_intrinsic_dim_and_noise(intrinsic_dim=5, noise_dim=15)

### a) Sklearn current heuristic — `'kd_tree'`

In [4]:
algo = choose_algo(X_train, heuristic='current')  # -> 'kd_tree'
model = NearestNeighbors(N_NEIGHBORS, algorithm=algo)

In [5]:
%%timeit

fit_predict(X_train, X_test, model)

3.26 s ± 143 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### b) Proposed heuristic — `'brute'`

In [6]:
algo = choose_algo(X_train, heuristic='proposed')  # -> 'brute'
model = NearestNeighbors(N_NEIGHBORS, algorithm=algo)

In [7]:
%%timeit

fit_predict(X_train, X_test, model)

34.1 s ± 123 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### c) Comments

The proposed heuristic is failing over the original because of the low intrinsic dimensionality of the data. Here by a factor of 10.46. Note that, the smaller `n_features` is, the faster `'kd_tree'` is over `'brute'`.

## II - High intrinsic dimensionality

In [8]:
X_train, X_test = get_standard_normal(n_features=100)
n_neighbors = 10

### a) Sklearn current heuristic — `'kd_tree'`

In [9]:
algo = choose_algo(X_train, heuristic='current')  # -> 'kd_tree'
model = NearestNeighbors(N_NEIGHBORS, algorithm=algo)

In [10]:
%%time

fit_predict(X_train, X_test, model)

CPU times: user 10min 15s, sys: 105 ms, total: 10min 15s
Wall time: 10min 15s


### b) Proposed heuristic — `'brute'`

In [11]:
algo = choose_algo(X_train, heuristic='proposed')  # -> 'brute'
model = NearestNeighbors(N_NEIGHBORS, algorithm=algo)

In [12]:
%%timeit

fit_predict(X_train, X_test, model)

38.7 s ± 1.95 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### c) Comments

The original heuristic is failing over the proposed one because of the high intrinsic dimensionality of the data. Here by a factor of 15.9.

## III - Conclusion

The advantages of the proposed heuristics over the current one are:
- it fails with low instrinsic dimensionality datasets, which are not the most common cases in ML-neighboring settings
- when it fails, it's at a less annoying order of magnitude than the current heuristic
- it chooses the optimal method in the most expensive settings (high number of features)